# Running queries to PostgreSQL with SQLAlchemy


SQLAlchemy is a fully fledged ORM. This notebook will cover all the topics we have seen before but in SQLAlchemy. That is:

- Basic queries with SQLAlchemy
- Upload of csv as a new table in database with SQLAlchemy
- Basic iteration of data from that table on a select * statement
- Basic GROUP BY queries
- avoid sql injection in SQLAlchemy with Objects
- Secting certain columns, rows BY LIMITING and skipping
- Doing ORDER BY
- Adding pagination to an iterable and fetch results from it with itertools
- Simple LEFT JOIN between the drinks table and another made up table with information per country with a 1-1 relationship
- LEFT join between the drinks table and another made up table with a 1-many relationship
- Complex LEFT JOIN by doing a many-many relationship between a made up table and drinks
- More complex LEFT JOIN with temporary table creation between drinks and the many-many relationship table, to do an aggregation.


In [1]:
!pip install sqlalchemy

## Setup

In [69]:
from sqlalchemy import create_engine, Column, Integer, String, Float, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship

DB_HOST = 'test-db-sql-class.cnct5qiopjti.us-east-1.rds.amazonaws.com'
DB_PORT = 5432
DB_NAME = 'students'
DB_USER = 'student'
DB_PASSWORD = 'Password123$'
STUDENT_NAME_TABLE = 'default'  # CHANGE THIS!

# Database Connection
engine = create_engine(f'postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}')
Session = sessionmaker(bind=engine)
session = Session()
Base = declarative_base()


<ipython-input-69-b04cf0620eba>:16: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [20]:
class Drink(Base):
    __tablename__ = f'drinks_{STUDENT_NAME_TABLE}'
    id = Column(Integer, primary_key=True)
    country = Column(String)
    beer_servings = Column(Integer)
    spirit_servings = Column(Integer)
    wine_servings = Column(Integer)
    total_litres_of_pure_alcohol = Column(Float)
    continent = Column(String)

    def __repr__(self):
        return f'<Drink {self.country}, Beer: {self.beer_servings}, Spirit: {self.spirit_servings}, Wine: {self.wine_servings}, Alcohol: {self.total_litres_of_pure_alcohol}, Continent: {self.continent}>'

## 0. Upload CSV

In [21]:
%%writefile get_data.sh

mkdir -p data
if [ ! -f ./data/drinks.csv ]; then
    wget -O data/drinks.csv https://www.dropbox.com/scl/fi/tkfdy0mq30g2t424hmn5o/drinks.csv?rlkey=jl8r4aw1o7y7b5au8icub20pn&dl=0
fi


Overwriting get_data.sh


In [22]:
!bash get_data.sh

In [23]:
import pandas as pd

drinks_df = pd.read_csv('data/drinks.csv')
drinks_df.to_sql(f'drinks_{STUDENT_NAME_TABLE}', engine, if_exists='replace', index=True, index_label='id')


193

## 1. Basic Queries with SQLAlchemy

In [24]:
for drink in session.query(Drink).all():
    print(drink)


<Drink Afghanistan, Beer: 0, Spirit: 0, Wine: 0, Alcohol: 0.0, Continent: AS>
<Drink Albania, Beer: 89, Spirit: 132, Wine: 54, Alcohol: 4.9, Continent: EU>
<Drink Algeria, Beer: 25, Spirit: 0, Wine: 14, Alcohol: 0.7, Continent: AF>
<Drink Andorra, Beer: 245, Spirit: 138, Wine: 312, Alcohol: 12.4, Continent: EU>
<Drink Angola, Beer: 217, Spirit: 57, Wine: 45, Alcohol: 5.9, Continent: AF>
<Drink Antigua & Barbuda, Beer: 102, Spirit: 128, Wine: 45, Alcohol: 4.9, Continent: None>
<Drink Argentina, Beer: 193, Spirit: 25, Wine: 221, Alcohol: 8.3, Continent: SA>
<Drink Armenia, Beer: 21, Spirit: 179, Wine: 11, Alcohol: 3.8, Continent: EU>
<Drink Australia, Beer: 261, Spirit: 72, Wine: 212, Alcohol: 10.4, Continent: OC>
<Drink Austria, Beer: 279, Spirit: 75, Wine: 191, Alcohol: 9.7, Continent: EU>
<Drink Azerbaijan, Beer: 21, Spirit: 46, Wine: 5, Alcohol: 1.3, Continent: EU>
<Drink Bahamas, Beer: 122, Spirit: 176, Wine: 51, Alcohol: 6.3, Continent: None>
<Drink Bahrain, Beer: 42, Spirit: 63, W

In [25]:
for drink in session.query(Drink).filter(Drink.beer_servings > 10):
    print(drink)


<Drink Albania, Beer: 89, Spirit: 132, Wine: 54, Alcohol: 4.9, Continent: EU>
<Drink Algeria, Beer: 25, Spirit: 0, Wine: 14, Alcohol: 0.7, Continent: AF>
<Drink Andorra, Beer: 245, Spirit: 138, Wine: 312, Alcohol: 12.4, Continent: EU>
<Drink Angola, Beer: 217, Spirit: 57, Wine: 45, Alcohol: 5.9, Continent: AF>
<Drink Antigua & Barbuda, Beer: 102, Spirit: 128, Wine: 45, Alcohol: 4.9, Continent: None>
<Drink Argentina, Beer: 193, Spirit: 25, Wine: 221, Alcohol: 8.3, Continent: SA>
<Drink Armenia, Beer: 21, Spirit: 179, Wine: 11, Alcohol: 3.8, Continent: EU>
<Drink Australia, Beer: 261, Spirit: 72, Wine: 212, Alcohol: 10.4, Continent: OC>
<Drink Austria, Beer: 279, Spirit: 75, Wine: 191, Alcohol: 9.7, Continent: EU>
<Drink Azerbaijan, Beer: 21, Spirit: 46, Wine: 5, Alcohol: 1.3, Continent: EU>
<Drink Bahamas, Beer: 122, Spirit: 176, Wine: 51, Alcohol: 6.3, Continent: None>
<Drink Bahrain, Beer: 42, Spirit: 63, Wine: 7, Alcohol: 2.0, Continent: AS>
<Drink Barbados, Beer: 143, Spirit: 173, 

In [35]:
results = session.query(Drink.country, Drink.beer_servings).all()
for name, content in results:
    print(name, content)


Afghanistan 0
Albania 89
Algeria 25
Andorra 245
Angola 217
Antigua & Barbuda 102
Argentina 193
Armenia 21
Australia 261
Austria 279
Azerbaijan 21
Bahamas 122
Bahrain 42
Bangladesh 0
Barbados 143
Belarus 142
Belgium 295
Belize 263
Benin 34
Bhutan 23
Bolivia 167
Bosnia-Herzegovina 76
Botswana 173
Brazil 245
Brunei 31
Bulgaria 231
Burkina Faso 25
Burundi 88
Cote d'Ivoire 37
Cabo Verde 144
Cambodia 57
Cameroon 147
Canada 240
Central African Republic 17
Chad 15
Chile 130
China 79
Colombia 159
Comoros 1
Congo 76
Cook Islands 0
Costa Rica 149
Croatia 230
Cuba 93
Cyprus 192
Czech Republic 361
North Korea 0
DR Congo 32
Denmark 224
Djibouti 15
Dominica 52
Dominican Republic 193
Ecuador 162
Egypt 6
El Salvador 52
Equatorial Guinea 92
Eritrea 18
Estonia 224
Ethiopia 20
Fiji 77
Finland 263
France 127
Gabon 347
Gambia 8
Georgia 52
Germany 346
Ghana 31
Greece 133
Grenada 199
Guatemala 53
Guinea 9
Guinea-Bissau 28
Guyana 93
Haiti 1
Honduras 69
Hungary 234
Iceland 233
India 9
Indonesia 5
Iran 0
Iraq 9


## 2. Basic Iteration of Data

In [26]:
for drink in session.query(Drink).filter(Drink.country.like('%gen%')):
    print(drink)


<Drink Argentina, Beer: 193, Spirit: 25, Wine: 221, Alcohol: 8.3, Continent: SA>


In [ ]:
for drink in session.query(Drink).filter(Drink.beer_servings < 50).limit(10):
    print(drink)


## 3. GROUP BY

In [27]:
from sqlalchemy import func

for name, count in session.query(Drink.continent, func.count(Drink.id)).group_by(Drink.continent):
    print(name, count)


None 23
SA 12
OC 16
AS 44
AF 53
EU 45


In [ ]:
for country, avg_wine in session.query(Drink.continent, func.avg(Drink.wine_servings)).group_by(Drink.continent):
    print(country, avg_wine)

## 4. Avoid Injection

In [28]:
query = session.query(Drink).filter(Drink.continent == 'EU')
query.all()

[<Drink Albania, Beer: 89, Spirit: 132, Wine: 54, Alcohol: 4.9, Continent: EU>,
 <Drink Andorra, Beer: 245, Spirit: 138, Wine: 312, Alcohol: 12.4, Continent: EU>,
 <Drink Armenia, Beer: 21, Spirit: 179, Wine: 11, Alcohol: 3.8, Continent: EU>,
 <Drink Austria, Beer: 279, Spirit: 75, Wine: 191, Alcohol: 9.7, Continent: EU>,
 <Drink Azerbaijan, Beer: 21, Spirit: 46, Wine: 5, Alcohol: 1.3, Continent: EU>,
 <Drink Belarus, Beer: 142, Spirit: 373, Wine: 42, Alcohol: 14.4, Continent: EU>,
 <Drink Belgium, Beer: 295, Spirit: 84, Wine: 212, Alcohol: 10.5, Continent: EU>,
 <Drink Bosnia-Herzegovina, Beer: 76, Spirit: 173, Wine: 8, Alcohol: 4.6, Continent: EU>,
 <Drink Bulgaria, Beer: 231, Spirit: 252, Wine: 94, Alcohol: 10.3, Continent: EU>,
 <Drink Croatia, Beer: 230, Spirit: 87, Wine: 254, Alcohol: 10.2, Continent: EU>,
 <Drink Cyprus, Beer: 192, Spirit: 154, Wine: 113, Alcohol: 8.2, Continent: EU>,
 <Drink Czech Republic, Beer: 361, Spirit: 170, Wine: 134, Alcohol: 11.8, Continent: EU>,
 <Dri

In [ ]:
beer = 50
query = session.query(Drink).filter_by(beer_servings>beer)
query.all()

## 5. Selecting Specific Columns, Rows, and Limiting/Skipping

In [30]:
for drink in session.query(Drink).order_by(Drink.spirit_servings.desc()).limit(5).offset(10):
    print(drink)


<Drink Bulgaria, Beer: 231, Spirit: 252, Wine: 94, Alcohol: 10.3, Continent: EU>
<Drink Kazakhstan, Beer: 124, Spirit: 246, Wine: 12, Alcohol: 6.8, Continent: AS>
<Drink Lithuania, Beer: 343, Spirit: 244, Wine: 56, Alcohol: 12.9, Continent: EU>
<Drink Ukraine, Beer: 206, Spirit: 237, Wine: 45, Alcohol: 8.9, Continent: EU>
<Drink Moldova, Beer: 109, Spirit: 226, Wine: 18, Alcohol: 6.3, Continent: EU>


In [ ]:
for drink in session.query(Drink.country).filter(Drink.beer_servings > 10).order_by(Drink.spirit_servings.desc()).limit(5).offset(5):
    print(drink)

## 6. ORDER BY queries

In [32]:
for drink in session.query(Drink).order_by(Drink.total_litres_of_pure_alcohol.desc(), Drink.country):
    print(drink.country, drink.total_litres_of_pure_alcohol)


Belarus 14.4
Lithuania 12.9
Andorra 12.4
Grenada 11.9
Czech Republic 11.8
France 11.8
Russian Federation 11.5
Ireland 11.4
Luxembourg 11.4
Slovakia 11.4
Germany 11.3
Hungary 11.3
Portugal 11.0
Poland 10.9
Slovenia 10.6
Belgium 10.5
Latvia 10.5
Australia 10.4
Denmark 10.4
Romania 10.4
United Kingdom 10.4
Bulgaria 10.3
Croatia 10.2
Switzerland 10.2
St. Lucia 10.1
Finland 10.0
Spain 10.0
South Korea 9.8
Austria 9.7
Serbia 9.6
Estonia 9.5
Netherlands 9.4
New Zealand 9.3
Nigeria 9.1
Gabon 8.9
Ukraine 8.9
USA 8.7
Argentina 8.3
Greece 8.3
Uganda 8.3
Canada 8.2
Cyprus 8.2
South Africa 8.2
St. Kitts & Nevis 7.7
Venezuela 7.7
Chile 7.6
Paraguay 7.3
Brazil 7.2
Panama 7.2
Sweden 7.2
Guyana 7.1
Japan 7.0
Niue 7.0
Palau 6.9
Belize 6.8
Kazakhstan 6.8
Namibia 6.8
Rwanda 6.8
Norway 6.7
Sierra Leone 6.7
Dominica 6.6
Iceland 6.6
Malta 6.6
Uruguay 6.6
Italy 6.5
Thailand 6.4
Trinidad & Tobago 6.4
Bahamas 6.3
Barbados 6.3
Burundi 6.3
Moldova 6.3
St. Vincent & the Grenadines 6.3
Dominican Republic 6.2
Laos 6

## 7. Pagination with itertools

In [34]:
from itertools import islice, zip_longest

def batched(iterable, n):
    "Batch data into lists of length n. The last batch may be shorter."
    # batched('ABCDEFG', 3) --> ABC DEF G
    it = iter(iterable)
    while True:
        batch = list(islice(it, n))
        if not batch:
            return
        yield batch


query = session.query(Drink).order_by(Drink.country)
page_size = 5

for page, result in enumerate(batched(query, page_size)):
    print(page, result)


0 [<Drink Afghanistan, Beer: 0, Spirit: 0, Wine: 0, Alcohol: 0.0, Continent: AS>, <Drink Albania, Beer: 89, Spirit: 132, Wine: 54, Alcohol: 4.9, Continent: EU>, <Drink Algeria, Beer: 25, Spirit: 0, Wine: 14, Alcohol: 0.7, Continent: AF>, <Drink Andorra, Beer: 245, Spirit: 138, Wine: 312, Alcohol: 12.4, Continent: EU>, <Drink Angola, Beer: 217, Spirit: 57, Wine: 45, Alcohol: 5.9, Continent: AF>]
1 [<Drink Antigua & Barbuda, Beer: 102, Spirit: 128, Wine: 45, Alcohol: 4.9, Continent: None>, <Drink Argentina, Beer: 193, Spirit: 25, Wine: 221, Alcohol: 8.3, Continent: SA>, <Drink Armenia, Beer: 21, Spirit: 179, Wine: 11, Alcohol: 3.8, Continent: EU>, <Drink Australia, Beer: 261, Spirit: 72, Wine: 212, Alcohol: 10.4, Continent: OC>, <Drink Austria, Beer: 279, Spirit: 75, Wine: 191, Alcohol: 9.7, Continent: EU>]
2 [<Drink Azerbaijan, Beer: 21, Spirit: 46, Wine: 5, Alcohol: 1.3, Continent: EU>, <Drink Bahamas, Beer: 122, Spirit: 176, Wine: 51, Alcohol: 6.3, Continent: None>, <Drink Bahrain, Be

## Setup for JOINs

In [70]:
from sqlalchemy import Table, Column, ForeignKey, Integer, String, Float, MetaData, Table, inspect, text

metadata = MetaData()

# Inspect the database to check for table existence
inspector = inspect(engine)
tables = ['dummy_drinks', 'countries', 'drink_reviews', 'drink_ingredients', 'ingredients', f'dummy_drinks_{STUDENT_NAME_TABLE}', f'countries_{STUDENT_NAME_TABLE}', f'drink_reviews_{STUDENT_NAME_TABLE}', f'drink_ingredients_{STUDENT_NAME_TABLE}', f'ingredients_{STUDENT_NAME_TABLE}']
for table in tables:
   with engine.connect() as connection:
      connection.execute(text(f"DROP TABLE IF EXISTS {table} CASCADE;"))


In [71]:
class Country(Base):
    __tablename__ = f'countries_{STUDENT_NAME_TABLE}'
    id = Column(Integer, primary_key=True)
    name = Column(String)

class DummyDrink(Base):
    __tablename__ = f'dummy_drinks_{STUDENT_NAME_TABLE}'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    alcohol_content = Column(Float)
    country_id = Column(Integer, ForeignKey(f'countries_{STUDENT_NAME_TABLE}.id'))
    reviews = relationship('DrinkReview', backref='drink')
    drink_ingredients = relationship('DrinkIngredient', back_populates='drink')  # Correct relationship name

class DrinkReview(Base):
    __tablename__ = f'drink_reviews_{STUDENT_NAME_TABLE}'
    id = Column(Integer, primary_key=True)
    drink_id = Column(Integer, ForeignKey(f'dummy_drinks_{STUDENT_NAME_TABLE}.id'))
    review = Column(String)

class Ingredient(Base):
    __tablename__ = f'ingredients_{STUDENT_NAME_TABLE}'
    id = Column(Integer, primary_key=True)
    name = Column(String)
    drink_ingredients = relationship('DrinkIngredient', back_populates='ingredient')  # Correct relationship name

class DrinkIngredient(Base):
    __tablename__ = f'drink_ingredients_{STUDENT_NAME_TABLE}'
    drink_id = Column(Integer, ForeignKey(f'dummy_drinks_{STUDENT_NAME_TABLE}.id'), primary_key=True)
    ingredient_id = Column(Integer, ForeignKey(f'ingredients_{STUDENT_NAME_TABLE}.id'), primary_key=True)
    drink = relationship('DummyDrink', back_populates='drink_ingredients')  # Match attribute in DummyDrink
    ingredient = relationship('Ingredient', back_populates='drink_ingredients')  # Match attribute in Ingredient


Base.metadata.create_all(engine)


In [72]:
import random
# Now populate the tables using the revised model name

# Populate Countries
countries = ['France', 'Germany', 'Italy', 'Spain', 'USA', 'Canada', 'Japan', 'Mexico', 'Brazil', 'Argentina']
for country_name in countries:
    country = Country(name=country_name)
    session.add(country)
session.commit()

# Populate Dummy Drinks
drink_names = [f'Drink {i}' for i in range(1, 21)]
for drink_name in drink_names:
    drink = DummyDrink(name=drink_name, alcohol_content=random.uniform(0, 40), country_id=random.randint(1, len(countries)))
    session.add(drink)
session.commit()

# Populate Drink Reviews
reviews = ['Good', 'Excellent', 'Fair', 'Poor', 'Great', 'Okay']
for _ in range(20):
    review = DrinkReview(drink_id=random.randint(1, 20), review=random.choice(reviews))
    session.add(review)
session.commit()

# Populate Ingredients
ingredients = ['Water', 'Barley', 'Yeast', 'Hops', 'Rice', 'Corn', 'Sugar', 'Grapes', 'Apple', 'Orange']
for ingredient_name in ingredients:
    ingredient = Ingredient(name=ingredient_name)
    session.add(ingredient)
session.commit()

# Populate Drink Ingredients Association
for drink_id in range(1, 21):
    ingredient_ids = random.sample(range(1, 11), k=random.randint(1, 4))
    for ingredient_id in ingredient_ids:
        assoc = {
            "drink_id": drink_id,
            "ingredient_id": ingredient_id
        }
        session.bulk_insert_mappings(DrinkIngredient, [assoc])

session.commit()

## 8. Left JOIN (1-1)

In [75]:
for drink, country in session.query(DummyDrink, Country).join(Country, DummyDrink.country_id == Country.id):
    print(drink.name, country.name)

Drink 1 France
Drink 2 Japan
Drink 3 USA
Drink 4 USA
Drink 5 France
Drink 6 Argentina
Drink 7 Japan
Drink 8 Italy
Drink 9 France
Drink 10 USA
Drink 11 USA
Drink 12 Italy
Drink 13 Canada
Drink 14 Germany
Drink 15 USA
Drink 16 USA
Drink 17 Germany
Drink 18 Brazil
Drink 19 France
Drink 20 Mexico


## 9. Left JOIN (1-Many)

In [77]:
for drink, review in session.query(DummyDrink, DrinkReview).join(DrinkReview, DummyDrink.id == DrinkReview.drink_id):
    print(drink.name, review.review if review else "No review")


Drink 4 Fair
Drink 9 Poor
Drink 15 Great
Drink 16 Great
Drink 18 Good
Drink 17 Great
Drink 13 Excellent
Drink 1 Good
Drink 20 Excellent
Drink 3 Good
Drink 6 Okay
Drink 1 Poor
Drink 15 Great
Drink 20 Poor
Drink 11 Excellent
Drink 7 Great
Drink 3 Excellent
Drink 8 Poor
Drink 10 Poor
Drink 13 Poor


## 10. Left JOIN (Many - Many)

In [82]:
for drink, ingredient in session.query(DummyDrink, Ingredient).\
        join(DrinkIngredient, DummyDrink.id == DrinkIngredient.drink_id).\
        join(Ingredient, DrinkIngredient.ingredient_id == Ingredient.id):
    print(drink.name, ingredient.name if ingredient else "No ingredient")


Drink 1 Hops
Drink 2 Yeast
Drink 3 Corn
Drink 3 Barley
Drink 3 Sugar
Drink 4 Barley
Drink 4 Yeast
Drink 4 Corn
Drink 5 Water
Drink 5 Grapes
Drink 5 Orange
Drink 6 Yeast
Drink 6 Hops
Drink 6 Rice
Drink 7 Water
Drink 7 Grapes
Drink 7 Yeast
Drink 8 Orange
Drink 8 Corn
Drink 8 Apple
Drink 9 Hops
Drink 9 Sugar
Drink 9 Water
Drink 10 Sugar
Drink 10 Grapes
Drink 10 Water
Drink 10 Hops
Drink 11 Yeast
Drink 11 Orange
Drink 12 Water
Drink 12 Barley
Drink 12 Apple
Drink 13 Corn
Drink 13 Apple
Drink 13 Hops
Drink 13 Orange
Drink 14 Grapes
Drink 14 Apple
Drink 15 Water
Drink 15 Sugar
Drink 15 Grapes
Drink 16 Grapes
Drink 16 Corn
Drink 17 Barley
Drink 17 Apple
Drink 18 Water
Drink 18 Barley
Drink 19 Yeast
Drink 20 Sugar
Drink 20 Grapes
Drink 20 Water


## 11. Left JOIN with Temp Tables

In [93]:
query = f"""
WITH drink_ingredient_count AS (
    SELECT ingredient_id, COUNT(*) AS count
    FROM drink_ingredients_{STUDENT_NAME_TABLE}
    GROUP BY ingredient_id
)
SELECT ingredients_{STUDENT_NAME_TABLE}.name, drink_ingredient_count.count
FROM ingredients_{STUDENT_NAME_TABLE}
LEFT JOIN drink_ingredient_count ON ingredients_{STUDENT_NAME_TABLE}.id = drink_ingredient_count.ingredient_id
ORDER BY drink_ingredient_count.count DESC
LIMIT 1;
"""
result = session.execute(text(query))
for row in result:
    print(row)


('Water', 8)


## Final Integrating Exercise

**Mini-Hackathon**

Develop a Flask application that serves as the backend for a blog. This task involves creating SQLAlchemy models for blog entities, performing CRUD operations, and exposing these through RESTful API endpoints.

**Key Components:**

1. Define SQLAlchemy models for blog entities (Post, Author, Comment).
1. Populate these models with dummy data.
1. Implement CRUD operations in Flask routes.
1. Complex JOIN operations to fetch and aggregate blog data.
1. Data visualization using Matplotlib or Seaborn, if applicable.

**Execution Steps:**

1. Set up a Flask app and define routes.
1. Integrate SQLAlchemy models within Flask.
1. Ensure each route performs the intended database operation.
1. Test the application thoroughly.

This comprehensive task tests a range of skills from database design to API development and offers an excellent opportunity to create a practical, functioning web application using SQLAlchemy and Flask.

In [95]:
!pip install flask flask-sqlalchemy flask-migrate flask-restful

In [ ]:
from flask import Flask, jsonify, request

app = Flask(__name__)

# SQLAlchemy setup would go here
# from your_model_file import YourModel

@app.route('/')
def index():
    return "Welcome to the Blog API!"

# Endpoint to fetch all posts (students to integrate SQLAlchemy)
@app.route('/posts', methods=['GET'])
def get_posts():
    # Fetch posts from database using SQLAlchemy
    # posts = YourModel.query.all()
    # Convert the posts to a JSON format
    # posts_json = [post.to_dict() for post in posts]
    # return jsonify(posts_json)
    return "Endpoint to show all posts."

# Endpoint to create a new post
@app.route('/posts', methods=['POST'])
def create_post():
    # Extract data from request
    # data = request.json
    # Create a new post using SQLAlchemy
    # new_post = YourModel(data)
    # Add and commit new_post to database
    # Return success message or new post data
    return "Endpoint to create a new post."

# Endpoint to fetch a single post by ID
@app.route('/posts/<int:post_id>', methods=['GET'])
def get_post(post_id):
    # Fetch the post with the given ID using SQLAlchemy
    # post = YourModel.query.get(post_id)
    # Convert the post to JSON format and return
    return f"Endpoint to show post with id {post_id}."

# Endpoint to update a post by ID
@app.route('/posts/<int:post_id>', methods=['PUT'])
def update_post(post_id):
    # Extract updated data from request
    # Update post in database using SQLAlchemy
    # Return success message or updated post data
    return f"Endpoint to update post with id {post_id}."

# Endpoint to delete a post by ID
@app.route('/posts/<int:post_id>', methods=['DELETE'])
def delete_post(post_id):
    # Delete the post with the given ID using SQLAlchemy
    # Return success message
    return f"Endpoint to delete post with id {post_id}."

if __name__ == '__main__':
    app.run(debug=True)
